### Analysis

In [1]:
import tobac
from iris.analysis import COUNT, MEDIAN, MEAN

In [64]:
%%capture

data = tobac.testing.make_sample_data_2D_3blobs_inv()
dxy, dt = tobac.utils.get_spacings(data)
threshold = 9
features = tobac.themes.tobac_v1.feature_detection_multithreshold(data, dxy, threshold)
mask, features = tobac.themes.tobac_v1.segmentation(features, data, dxy, threshold=9)
track = tobac.themes.tobac_v1.linking_trackpy(features,data, dt=dt,dxy=dxy, v_max=100)

In [68]:
track

<xarray.Dataset>
Dimensions:                  (index: 102)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 5 6 ... 96 97 98 99 100 101
Data variables: (12/16)
    frame                    (index) int64 0 1 2 3 4 5 6 ... 67 67 68 68 69 69
    idx                      (index) int64 1 1 1 1 1 1 1 1 1 ... 1 2 1 2 1 2 1 2
    hdim_1                   (index) float64 10.0 11.85 13.66 ... 90.79 93.52
    hdim_2                   (index) float64 10.0 10.94 11.71 ... 30.92 132.7
    num                      (index) int64 69 66 65 65 65 ... 495 278 412 229
    threshold_value          (index) int64 9 9 9 9 9 9 9 9 9 ... 9 9 9 9 9 9 9 9
    ...                       ...
    projection_y_coordinate  (index) float64 1e+04 1.094e+04 ... 1.327e+05
    latitude                 (index) object 24.1 24.12 24.14 ... 24.91 24.94
    longitude                (index) object 150.1 150.1 150.1 ... 150.3 151.3
    ncells                   (index) float64 69.0 67.0 65.0 ... 229.0 229.0
    cell                     (index) float64 1.0 1.0 1.0 1.0 ... 2.0 3.0 2.0 3.0
    time_cell                (index) timedelta64[ns] 00:00:00 ... 00:19:00

In [69]:
tobac.plot.plot_tracks_mask_field(track, data, mask, features)

ValueError: axes had to be cartopy.mpl.geoaxes.GeoAxesSubplot

In [37]:
track

<xarray.Dataset>
Dimensions:                  (index: 30)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 5 6 ... 24 25 26 27 28 29
Data variables: (12/16)
    frame                    (index) int64 0 1 2 3 4 5 6 ... 22 22 23 23 24 24
    idx                      (index) int64 1 1 1 1 1 1 1 1 1 ... 1 2 1 2 1 2 1 2
    hdim_1                   (index) float64 10.0 11.71 13.2 ... 15.78 50.27
    hdim_2                   (index) float64 10.0 13.66 17.11 ... 25.55 95.97
    num                      (index) int64 69 65 65 66 65 ... 66 500 65 504 59
    threshold_value          (index) int64 9 9 9 9 9 9 9 9 9 ... 9 9 9 9 9 9 9 9
    ...                       ...
    projection_x_coordinate  (index) float64 1e+04 1.366e+04 ... 9.597e+04
    latitude                 (index) object 24.1 24.14 24.17 ... 24.26 24.96
    longitude                (index) object 150.1 150.1 150.1 ... 150.2 150.5
    ncells                   (index) int64 225 203 201 206 ... 201 201 189 189
    cell                     (index) float64 1.0 1.0 1.0 1.0 ... 2.0 1.0 2.0 1.0
    time_cell                (index) timedelta64[ns] 00:00:00 ... 00:48:00

In [38]:
data.to_iris()

W (m s-1),time,geopotential_height,projection_y_coordinate,projection_x_coordinate
Shape,25,20,200,100
Dimension coordinates,,,,
time,x,-,-,-
geopotential_height,-,x,-,-
projection_y_coordinate,-,-,x,-
projection_x_coordinate,-,-,-,x
Auxiliary coordinates,,,,
latitude,-,-,x,x
longitude,-,-,x,x


In [39]:
%%capture

data = tobac.testing.make_sample_data_3D_3blobs()
dxy, dt = tobac.utils.get_spacings(data)
threshold = 9
features = tobac.themes.tobac_v1.feature_detection_multithreshold(data.max('z'), dxy, threshold)
mask, features = tobac.themes.tobac_v1.segmentation(features, data, dxy, threshold=9)
track = tobac.themes.tobac_v1.linking_trackpy(features,data, dt=dt,dxy=dxy, v_max=100)

In [61]:
mask

<xarray.DataArray 'segmentation_mask' (time: 25, z: 20, y: 200, x: 100)>
[10000000 values with dtype=int32]
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01T12:00:00 ... 2000-01-01T12:48:00
  * z          (z) float64 0.0 1e+03 2e+03 3e+03 ... 1.7e+04 1.8e+04 1.9e+04
  * y          (y) float64 0.0 1e+03 2e+03 3e+03 ... 1.97e+05 1.98e+05 1.99e+05
  * x          (x) float64 0.0 1e+03 2e+03 3e+03 ... 9.7e+04 9.8e+04 9.9e+04
    latitude   (y, x) float64 ...
    longitude  (y, x) float64 ...
Attributes:
    long_name:  segmentation_mask

In [62]:
tobac.analysis.cell_statistics(data, track, mask, [MEAN, COUNT], 1,  z_coord='geopotential_height')

/home/nils/anaconda3/envs/tob_v2/lib/python3.7/site-packages/numpy/ma/core.py:3385: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  _data[indx] = dval


IndexError: too many indices for array: array is 3-dimensional, but 40 were indexed

In [54]:
import pandas as pd
import numpy as np
import logging
import os

from tobac.utils import mask_cell, mask_cell_surface, mask_cube_cell, get_bounding_box
from tobac.utils.convert import (
    xarray_to_iris,
    iris_to_xarray,
    xarray_to_irispandas,
    irispandas_to_xarray,
)
@xarray_to_irispandas
def cell_statistics(
    input_cubes,
    track,
    mask,
    aggregators,
    cell,
    output_path="./",
    output_name="Profiles",
    width=10000,
    z_coord="model_level_number",
    dimensions=["x", "y"],
    **kwargs
):
    from iris.cube import Cube, CubeList
    from iris.coords import AuxCoord
    from iris import Constraint, save

    # If input is single cube, turn into cubelist
    if type(input_cubes) is Cube:
        input_cubes = CubeList([input_cubes])

    logging.debug("Start calculating profiles for cell " + str(cell))
    track_i = track[track["cell"] == cell]

    cubes_profile = {}
    for aggregator in aggregators:
        cubes_profile[aggregator.name()] = CubeList()

    for time_i in track_i["time"].values:
        constraint_time = Constraint(time=time_i)

        mask_i = mask.extract(constraint_time)
        mask_cell_i = mask_cell(mask_i, cell, track_i, masked=False)
        mask_cell_surface_i = mask_cell_surface(
            mask_i, cell, track_i, masked=False, z_coord=z_coord
        )

        x_dim = mask_cell_surface_i.coord_dims("projection_x_coordinate")[0]
        y_dim = mask_cell_surface_i.coord_dims("projection_y_coordinate")[0]
        x_coord = mask_cell_surface_i.coord("projection_x_coordinate")
        y_coord = mask_cell_surface_i.coord("projection_y_coordinate")

        if (mask_cell_surface_i.core_data() > 0).any():
            box_mask_i = get_bounding_box(mask_cell_surface_i.core_data(), buffer=1)

            box_mask = [
                [
                    x_coord.points[box_mask_i[x_dim][0]],
                    x_coord.points[box_mask_i[x_dim][1]],
                ],
                [
                    y_coord.points[box_mask_i[y_dim][0]],
                    y_coord.points[box_mask_i[y_dim][1]],
                ],
            ]
        else:
            box_mask = [[np.nan, np.nan], [np.nan, np.nan]]

        x = track_i[track_i["time"].values == time_i]["projection_x_coordinate"].values[
            0
        ]
        y = track_i[track_i["time"].values == time_i]["projection_y_coordinate"].values[
            0
        ]

        box_slice = [[x - width, x + width], [y - width, y + width]]

        x_min = np.nanmin([box_mask[0][0], box_slice[0][0]])
        x_max = np.nanmax([box_mask[0][1], box_slice[0][1]])
        y_min = np.nanmin([box_mask[1][0], box_slice[1][0]])
        y_max = np.nanmax([box_mask[1][1], box_slice[1][1]])

        constraint_x = Constraint(
            projection_x_coordinate=lambda cell: int(x_min) < cell < int(x_max)
        )
        constraint_y = Constraint(
            projection_y_coordinate=lambda cell: int(y_min) < cell < int(y_max)
        )

        constraint = constraint_time & constraint_x & constraint_y
        #       Mask_cell_surface_i=mask_cell_surface(Mask_w_i,cell,masked=False,z_coord='model_level_number')
        mask_cell_i = mask_cell_i.extract(constraint)
        mask_cell_surface_i = mask_cell_surface_i.extract(constraint)

        input_cubes_i = input_cubes.extract(constraint)
        for cube in input_cubes_i:
            cube_masked = mask_cube_cell(cube, mask_cell_i, cell, track_i)
            coords_remove = []
            for coordinate in cube_masked.coords(dim_coords=False):

                if coordinate.name() not in dimensions:
                    for dim in dimensions:
                        if set(cube_masked.coord_dims(coordinate)).intersection(
                            set(cube_masked.coord_dims(dim))
                        ):
                            coords_remove.append(coordinate.name())
            for coordinate in set(coords_remove):
                cube_masked.remove_coord(coordinate)

            for aggregator in aggregators:
                cube_collapsed = cube_masked.collapsed(dimensions, aggregator, **kwargs)
                # remove all collapsed coordinates (x and y dim, scalar now) and keep only time as all these coordinates are useless
                for coordinate in cube_collapsed.coords():
                    if not cube_collapsed.coord_dims(coordinate):
                        if coordinate.name() != "time":
                            cube_collapsed.remove_coord(coordinate)
                logging.debug(str(cube_collapsed))
                cubes_profile[aggregator.name()].append(cube_collapsed)

    minutes = (track_i["time_cell"] / pd.Timedelta(minutes=1)).values
    latitude = track_i["latitude"].values
    longitude = track_i["longitude"].values
    minutes_coord = AuxCoord(minutes, long_name="cell_time", units="min")
    latitude_coord = AuxCoord(latitude, long_name="latitude", units="degrees")
    longitude_coord = AuxCoord(longitude, long_name="longitude", units="degrees")

    for aggregator in aggregators:
        cubes_profile[aggregator.name()] = cubes_profile[aggregator.name()].merge()
        for cube in cubes_profile[aggregator.name()]:
            cube.add_aux_coord(minutes_coord, data_dims=cube.coord_dims("time"))
            cube.add_aux_coord(latitude_coord, data_dims=cube.coord_dims("time"))
            cube.add_aux_coord(longitude_coord, data_dims=cube.coord_dims("time"))
        os.makedirs(
            os.path.join(output_path, output_name, aggregator.name()), exist_ok=True
        )
        savefile = os.path.join(
            output_path,
            output_name,
            aggregator.name(),
            output_name + "_" + aggregator.name() + "_" + str(int(cell)) + ".nc",
        )
        save(cubes_profile[aggregator.name()], savefile)

In [56]:
cell_statistics(data.to_iris(), track, mask, [MEAN], 2,  z_coord='z')

/home/nils/anaconda3/envs/tob_v2/lib/python3.7/site-packages/numpy/ma/core.py:3385: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  _data[indx] = dval


IndexError: too many indices for array: array is 3-dimensional, but 40 were indexed

In [49]:
track.to_dask_dataframe()[track.to_dataframe()["cell"] == 1]

/home/nils/anaconda3/envs/tob_v2/lib/python3.7/site-packages/dask/dataframe/core.py:4229: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  meta = self._meta[_extract_meta(key)]


,index,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,ncells,cell,time_cell
npartitions=1,,,,,,,,,,,,,,,,,
0,int64,int64,int64,float64,float64,int64,int64,int64,object,object,float64,float64,object,object,int64,float64,timedelta64[ns]
29,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
